In [1]:
import FedUtils
from model.utils.config import cfg

### Arguments

In [2]:
class Arguments():
    def __init__(self):
        self.class_agnostic =False
        self.epochs = 3
        self.lr = 0.001
        self.lr_decay_step = 4
        self.dataset = 'KAIST'
        self.net = 'vgg16'
        self.batch_size = 24
        self.nw = 2
        self.cuda = True
        #self.round = 10
        self.mGPUs=True
        self.optimizer ="sgd"
        self.k=3
        #self.device = th.device("cpu")
        
args = Arguments()

### load model

In [12]:
round_i = 10
parties = 2
sub_folder = 'wkFedPer_cd'

model_path = ['models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_campus_'+str(round_i)+'_3_718.pth',
#'models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_road_'+str(round_i)+'_3_513.pth',
'models/vgg16/KAIST/'+sub_folder+'/faster_rcnn_KAIST_downtown_'+str(round_i)+'_3_566.pth']


imdb_classes =  ('__background__',  # always index 0
                          'person',
                          'people','cyclist'
                         )

model_list=[None] * parties
for i in range(parties):
    model_list[i],optimizer, start_round = FedUtils.load_model(imdb_classes,model_path[i], args,cfg)

Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth
Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth


In [4]:
def FedPer(model_list,ratio_list,mGPUs):
    
    model_tmp=[None] * parties
    #optims_tmp=[None] * parties

    for idx, my_model in enumerate(model_list):
        if mGPUs:
            my_model = my_model.module
            
        model_tmp[idx] = my_model.RCNN_base.state_dict()


    for key in model_tmp[0]:    
        #print(key)
        model_avg = 0

        for idx, model_tmp_content in enumerate(model_tmp):     # add each model              
            model_avg += ratio_list[idx] * model_tmp_content[key]
            
        for i in range(len(model_tmp)):  #copy to each model            
            model_tmp[i][key] = model_avg
    #copy back to original model
    for i in range(len(model_list)):  
        if mGPUs:
            model_list[i].module.RCNN_base.load_state_dict(model_tmp[i])
        else:
            model_list[i].RCNN_base.load_state_dict(model_tmp[i])
    return model_list

### load test image

In [5]:
from frcnn_helper import *


        
def getWeight(test_images,model_list, args):
    
    wk_list = []
    for fasterRCNN in model_list:
        if args.mGPUs:
            fasterRCNN = fasterRCNN.module
        X = get_features(fasterRCNN, test_images, args.batch_size)/255.0
        wk_value = within_cluster_dispersion(X, n_cluster=args.k)
        wk_list.append(wk_value)
        print(wk_value)
    
    return wk_list

In [6]:
import pickle
testimg_pickle_path = 'testimg2252.pkl'
with open(testimg_pickle_path, 'rb') as handle:
    test_images = pickle.load(handle)

In [14]:
round_i

10

In [13]:
wk_list_curr = getWeight(test_images,model_list, args)

0.7072552804704895
0.8670191545408553


In [15]:
from scipy.special import softmax
       
wk_list_prev=[0.748743138,0.97777546]

if round_i==1:
    wk_diff = wk_list_curr
else:
    wk_diff=[]
    for list1_c, list2_p in zip(wk_list_prev, wk_list_curr):        
        wk_diff.append(list1_c-list2_p)

print('diff={}'.format(wk_diff))

wk_ratio = softmax(wk_diff).tolist()    
print('wk_ratio={}'.format(wk_ratio))
wk_list_prev = wk_list_curr

diff=[0.04148785752951056, 0.11075630545914472]
wk_ratio=[0.4826898088256425, 0.5173101911743575]


In [16]:
model_list =FedPer(model_list,wk_ratio,args.mGPUs)

In [17]:
output_dir = "models/" + args.net + "/" + args.dataset + "/" + sub_folder
output_dir

'models/vgg16/KAIST/wkFedPer_cd'

In [18]:
import os
from model.utils.net_utils import save_checkpoint
save_name = os.path.join(output_dir, 'faster_rcnn_KAIST_AVG_{}.pth'.format(round_i))
save_checkpoint({
  #'session': 1,
  'round': round_i,
  'model':  model_list[0].module.state_dict() if args.mGPUs else model_list[0].state_dict(), 
  'optimizer': optimizer.state_dict(),
  'pooling_mode': cfg.POOLING_MODE,
  'class_agnostic': args.class_agnostic,
}, save_name)

## resume

* already got  
    * models/vgg16/KAIST/wkFedPer_cd/faster_rcnn_KAIST_campus_10_3_718.pth
    * models/vgg16/KAIST/wkFedPer_cd/faster_rcnn_KAIST_AVG_9.pth

* want to train downtown_10_3  from downtown_9(personal) +AVG (base layer)

### load model

* load AVG_9

In [6]:
load_name = 'models/vgg16/KAIST/wkFedPer_cd/faster_rcnn_KAIST_AVG_9.pth'
model_avg, optimizer, start_round =FedUtils.load_model(imdb_classes, load_name, args, cfg)

Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth


* load downtown_9_3

In [7]:
load_name = 'models/vgg16/KAIST/wkFedPer_cd/faster_rcnn_KAIST_downtown_9_3_566.pth'
model_downtown, optimizer, start_round =FedUtils.load_model(imdb_classes, load_name, args, cfg)

Loading pretrained weights from data/pretrained_model/vgg16_caffe.pth


In [8]:
optimizer = FedUtils.getOptimizer(model_downtown,args,cfg)

### combine model

In [11]:
model_tmp = model_avg.module.RCNN_base.state_dict()

In [13]:
model_downtown.module.RCNN_base.load_state_dict(model_tmp)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

### train

In [17]:
import os, pickle
from roi_data_layer.roidb import  filter_roidb

imdb_name = 'KAIST_downtown'
data_cache_path = 'data/cache'

pkl_file = os.path.join(data_cache_path, imdb_name + '_gt_roidb.pkl')

with open(pkl_file, 'rb') as f:
    roidb = pickle.load(f)

roidb = filter_roidb(roidb)

ratio_list, ratio_index = rank_roidb_ratio(roidb)

train_size = len(roidb)
print(train_size)
iters_per_epoch = int(train_size / args.batch_size)
print('iters_per_epoch: ' + str(iters_per_epoch))
iter_epochs_list.append(iters_per_epoch)
sampler_batch = sampler(train_size, args.batch_size)

dataset = roibatchLoader(roidb, ratio_list, ratio_index, args.batch_size, imdb_classes, training=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=args.batch_size,
                                         sampler=sampler_batch, num_workers=args.num_workers)
        

ImportError: cannot import name '_mask'

In [ ]:
train(args,dataloader,imdb_name,iters_per_epoch, fasterRCNN, optimizer, num_round):

# backup

In [6]:
wk_ratio = [0.33333,0.33333,0.33333]   

model_list = FedUtils.avgWeight(model_list,wk_ratio)


In [14]:
ratio_list = [0.33333,0.33333,0.33333]   

model_list = FedPer(model_list,ratio_list,True)

In [10]:
model_list[0].load_state_dict
for key in model_list[0].state_dict():
    print(key)

module.RCNN_rpn.RPN_Conv.weight
module.RCNN_rpn.RPN_Conv.bias
module.RCNN_rpn.RPN_cls_score.weight
module.RCNN_rpn.RPN_cls_score.bias
module.RCNN_rpn.RPN_bbox_pred.weight
module.RCNN_rpn.RPN_bbox_pred.bias
module.RCNN_base.0.weight
module.RCNN_base.0.bias
module.RCNN_base.2.weight
module.RCNN_base.2.bias
module.RCNN_base.5.weight
module.RCNN_base.5.bias
module.RCNN_base.7.weight
module.RCNN_base.7.bias
module.RCNN_base.10.weight
module.RCNN_base.10.bias
module.RCNN_base.12.weight
module.RCNN_base.12.bias
module.RCNN_base.14.weight
module.RCNN_base.14.bias
module.RCNN_base.17.weight
module.RCNN_base.17.bias
module.RCNN_base.19.weight
module.RCNN_base.19.bias
module.RCNN_base.21.weight
module.RCNN_base.21.bias
module.RCNN_base.24.weight
module.RCNN_base.24.bias
module.RCNN_base.26.weight
module.RCNN_base.26.bias
module.RCNN_base.28.weight
module.RCNN_base.28.bias
module.RCNN_top.0.weight
module.RCNN_top.0.bias
module.RCNN_top.3.weight
module.RCNN_top.3.bias
module.RCNN_cls_score.weight
m

In [9]:
model_list[0].module

vgg16(
  (RCNN_rpn): _RPN(
    (RPN_Conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (RPN_cls_score): Conv2d(512, 24, kernel_size=(1, 1), stride=(1, 1))
    (RPN_bbox_pred): Conv2d(512, 48, kernel_size=(1, 1), stride=(1, 1))
    (RPN_proposal): _ProposalLayer()
    (RPN_anchor_target): _AnchorTargetLayer()
  )
  (RCNN_proposal_target): _ProposalTargetLayer()
  (RCNN_roi_pool): ROIPool(output_size=(7, 7), spatial_scale=0.0625)
  (RCNN_roi_align): ROIAlign(output_size=(7, 7), spatial_scale=0.0625, sampling_ratio=0)
  (RCNN_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), 

In [22]:
model_list[0].module.RCNN_base

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [16]:
for l in list(model_list[0].module.RCNN_base.modules())[0]:
    print(l)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(512, 512, kernel_size=(3, 3), stride

In [34]:
ratio_list = [0.33333,0.33333,0.33333]   
model_tmp=[None] * parties


for idx, my_model in enumerate(model_list):
    model_tmp[idx] = my_model.module.RCNN_base.state_dict()
    
for key in model_tmp[0]:  
    print('key={}'.format(key))
    model_avg = 0
    for idx, model_tmp_content in enumerate(model_tmp):     # add each model              
        #print(model_tmp_content)
        model_avg += ratio_list[idx] * model_tmp_content[key]
            
        #for i in range(len(model_tmp)):  #copy to each model            
        #    model_tmp[i][key] = model_avg

key=0.weight
key=0.bias
key=2.weight
key=2.bias
key=5.weight
key=5.bias
key=7.weight
key=7.bias
key=10.weight
key=10.bias
key=12.weight
key=12.bias
key=14.weight
key=14.bias
key=17.weight
key=17.bias
key=19.weight
key=19.bias
key=21.weight
key=21.bias
key=24.weight
key=24.bias
key=26.weight
key=26.bias
key=28.weight
key=28.bias


In [5]:
for model in model_list:
    count=0
    for key in model.module.RCNN_base.state_dict():
        print(key)
        
        if count>10:
            print(model.module.RCNN_base.state_dict()[key])
        elif count>15:
            break
        count+=1
    

0.weight
0.bias
2.weight
2.bias
5.weight
5.bias
7.weight
7.bias
10.weight
10.bias
12.weight
12.bias
tensor([ 1.9681e-02, -1.3799e-03,  9.2438e-02,  1.2743e-01,  1.9292e-01,
         9.0432e-02,  6.8026e-02, -1.0927e-02, -5.1111e-02,  3.7352e-02,
        -1.4686e-02,  3.6139e-02,  4.4801e-02,  1.1423e-01,  3.9641e-02,
         3.3096e-02, -4.1428e-02,  1.2618e-01,  9.1683e-02,  1.1846e-01,
        -1.3171e-02,  7.5038e-02,  1.2834e-01,  8.0052e-03, -1.0758e-01,
         1.3260e-03, -3.4063e-02,  1.1694e-01,  2.8742e-02,  3.0856e-02,
         3.8783e-02, -6.8109e-02, -7.1249e-03,  1.0495e-02,  2.0629e-02,
         3.3418e-02,  4.5786e-03, -5.2766e-02,  4.8309e-02, -5.4794e-02,
         1.6837e-01,  1.1577e-02,  2.0177e-02,  1.6813e-02,  7.0814e-02,
         2.1722e-01,  1.5303e-03,  1.6412e-01,  3.0625e-03,  1.1699e-02,
        -5.8547e-03,  5.1910e-02, -5.6160e-02,  8.7344e-02,  1.5881e-02,
         8.7724e-02,  1.3232e-01,  3.8183e-02, -1.8726e-02, -6.9276e-02,
         5.7053e-02,  9.

tensor([ 3.5411e-02,  1.0791e-01, -1.8003e-02,  4.3315e-02, -5.5975e-02,
         3.7676e-02,  7.4026e-02,  1.1679e-02, -4.1718e-02, -1.1741e-01,
         6.7841e-02,  1.2219e-01,  2.0512e-02,  3.7287e-02, -1.7611e-02,
        -5.2910e-02,  1.3105e-01, -2.6485e-02, -3.4485e-04, -1.7344e-02,
        -4.9295e-04, -2.9398e-02, -1.8024e-02,  3.4394e-02,  1.0530e-01,
        -3.0907e-02, -5.6342e-02, -2.9152e-02, -5.6464e-02,  1.7369e-01,
         2.1761e-02,  2.0427e-01,  1.7811e-02, -2.0831e-02,  3.5783e-02,
        -3.3961e-02,  4.1354e-03,  5.3230e-02,  6.1176e-02, -2.2459e-02,
         2.5385e-02,  7.2709e-03,  1.3982e-01,  1.3646e-01, -4.5961e-02,
         1.1535e-02,  3.2299e-02, -1.4289e-01, -1.2844e-03, -2.5035e-02,
         9.6844e-02, -8.8837e-02,  4.1076e-02, -3.0009e-02,  2.8188e-01,
         5.0557e-02, -1.3483e-02,  5.9306e-02, -2.9203e-02,  9.0944e-03,
         2.9104e-03,  7.3505e-02, -6.0095e-03, -4.7786e-02,  7.3688e-03,
        -6.6424e-02, -6.0932e-02,  3.6776e-01, -1.1

tensor([[[[ 6.8152e-03,  6.3507e-03,  1.8901e-03],
          [ 5.1260e-03,  6.0116e-03,  2.4189e-03],
          [ 8.4551e-03,  9.0430e-03,  6.1686e-03]],

         [[ 1.0485e-03, -1.2216e-02, -1.5350e-02],
          [ 2.3707e-03,  1.7852e-03, -3.5234e-03],
          [ 3.7787e-03,  3.8959e-03, -2.0271e-03]],

         [[-4.8368e-03, -2.2648e-03, -7.2266e-03],
          [-9.1134e-03, -2.5107e-03, -1.0685e-02],
          [-1.2249e-02, -5.3186e-03, -4.4538e-03]],

         ...,

         [[ 1.2870e-03,  9.3651e-03,  4.2367e-03],
          [-2.9680e-03,  6.5011e-03,  4.6937e-03],
          [ 3.4670e-04,  3.4159e-03,  4.4858e-03]],

         [[-4.7826e-03, -3.2445e-03, -7.7611e-04],
          [-4.9114e-03, -1.1914e-02, -8.8670e-03],
          [ 3.1719e-03, -5.9615e-03,  9.1111e-04]],

         [[ 1.9188e-03,  7.0805e-04,  5.5216e-03],
          [-8.5219e-03, -1.3144e-02, -1.1819e-02],
          [ 2.3179e-03, -6.1564e-03, -8.0755e-03]]],


        [[[ 6.1528e-04, -5.6391e-03, -9.2436e-03],
  

tensor([[[[-2.8279e-04, -1.7617e-03,  3.5949e-03],
          [ 5.0466e-03, -6.4154e-04, -2.2147e-03],
          [-9.8392e-03, -6.0277e-03,  9.8647e-04]],

         [[-4.7531e-03, -1.2324e-03,  1.0371e-02],
          [-1.9017e-02, -5.9638e-03,  1.5037e-02],
          [-1.5857e-02, -6.9450e-03,  5.3497e-03]],

         [[ 8.0424e-03, -3.3744e-03, -1.6122e-03],
          [ 5.6232e-03, -9.2704e-03, -6.2996e-05],
          [ 1.3902e-02, -1.4453e-03,  7.1728e-04]],

         ...,

         [[ 2.0021e-03, -5.7104e-04, -8.6082e-03],
          [ 2.9558e-04, -3.6942e-03, -6.2007e-03],
          [ 8.8091e-03,  3.9101e-03, -1.3346e-03]],

         [[ 7.0985e-03, -1.3938e-02, -3.8066e-02],
          [ 3.5493e-03,  9.5193e-03, -1.4754e-02],
          [-9.3212e-03, -8.3984e-03, -1.6255e-02]],

         [[ 6.6508e-03,  1.0387e-02,  7.4136e-03],
          [ 4.3361e-04,  3.7124e-03,  5.4175e-03],
          [ 3.4076e-03,  5.6737e-03,  4.4267e-03]]],


        [[[ 1.3647e-03, -3.0235e-03, -1.3536e-03],
  

tensor([ 0.0135,  0.0076,  0.0670,  0.0418,  0.0013,  0.1943, -0.0424,  0.0275,
         0.0720,  0.0540,  0.0548,  0.0005,  0.0117,  0.0733,  0.0647, -0.0173,
         0.0117, -0.0091,  0.0594,  0.0078,  0.1947, -0.0316,  0.0823,  0.0081,
        -0.0109,  0.1183, -0.0016, -0.0310,  0.1052,  0.0891, -0.0175,  0.0111,
         0.0273,  0.0363,  0.1165,  0.0066,  0.0531, -0.0711,  0.0240,  0.0547,
         0.0187, -0.0151,  0.0688,  0.0197, -0.0144,  0.1044,  0.0295,  0.0108,
         0.1246,  0.0035, -0.0739, -0.0102,  0.0137,  0.0572, -0.0105,  0.0623,
         0.0447,  0.0586,  0.0430,  0.0326, -0.0471,  0.0135, -0.0043,  0.0153,
        -0.0114,  0.0794,  0.0746,  0.0432,  0.0419, -0.0083,  0.1241,  0.0263,
         0.0150, -0.0156,  0.1355, -0.0236,  0.1011,  0.0222,  0.0468, -0.0231,
        -0.0004,  0.0089,  0.0174,  0.0072, -0.0093,  0.0147,  0.0377, -0.0181,
        -0.0138,  0.0371, -0.0514,  0.0245,  0.0247,  0.0491,  0.0520,  0.1023,
         0.0564, -0.0158, -0.0332, -0.05

tensor([[[[-4.1516e-03, -3.9639e-03, -2.8086e-03],
          [-4.4497e-03, -5.4828e-03, -3.5015e-03],
          [-4.3465e-03, -4.6984e-03, -3.8301e-03]],

         [[-8.9229e-03, -4.5028e-03, -2.7504e-04],
          [-6.1409e-03, -1.7167e-03, -7.0691e-04],
          [-6.7249e-03, -1.8555e-03, -6.7953e-04]],

         [[-3.1488e-03, -2.5744e-03,  4.8868e-03],
          [-4.2171e-03, -5.5657e-03,  3.5389e-04],
          [-3.7091e-03, -5.8184e-03, -3.2052e-03]],

         ...,

         [[-4.3691e-03, -7.8000e-03, -5.2038e-03],
          [-1.3593e-02, -1.3360e-02, -1.0261e-02],
          [-1.2075e-02, -1.3351e-02, -1.1710e-02]],

         [[ 3.9933e-03,  1.0583e-03, -5.9391e-03],
          [ 2.3837e-03, -1.0948e-03, -1.0242e-02],
          [ 2.3343e-03,  1.2238e-03, -3.8519e-03]],

         [[-9.3036e-03, -7.8964e-03, -4.5588e-03],
          [-9.0906e-03, -5.8591e-03, -3.1243e-03],
          [-7.1458e-03, -2.1444e-03, -1.0698e-03]]],


        [[[ 7.3478e-03, -1.5052e-03,  2.3749e-03],
  